In [ ]:
%%capture
!pip install langchain langchain_community pypdf faiss-cpu

In [ ]:
%%capture
!pip install langchain_groq


In [ ]:
#loading the data
from langchain_community.document_loaders import PyPDFLoader
loader=PyPDFLoader("/content/Build_a_Large_Language_Model_From_Scratch_-_Sebastian_Raschka-1-10.pdf")
pages=loader.load()

print(pages[0])

page_content='MANNING
Sebastian Raschka
FROMSCRATCH
BUILD A' metadata={'source': '/content/Build_a_Large_Language_Model_From_Scratch_-_Sebastian_Raschka-1-10.pdf', 'page': 0, 'page_label': '1'}


In [ ]:
from langchain.docstore.document import Document
all_text="".join([page.page_content for page in pages])

dataset=Document(page_content=all_text,metadata={"source":"/content/Build_a_Large_Language_Model_From_Scratch_-_Sebastian_Raschka.pdf"})

In [ ]:
len(dataset.page_content)

5732

In [ ]:
## chunking
from langchain.text_splitter import CharacterTextSplitter,RecursiveCharacterTextSplitter
text_splitter=RecursiveCharacterTextSplitter(chunk_size=600,chunk_overlap=200)
chunks_data=text_splitter.split_documents([dataset])



In [ ]:
len(chunks_data)

14

In [ ]:
#creating vectorstor
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceBgeEmbeddings
model_name = "sentence-transformers/all-MiniLM-L6-v2" # Smaller model
hf_em_model=HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs={"device":"cpu"},
    encode_kwargs={"normalize":True}
)
vector_store=FAISS.from_documents(
    documents=chunks_data,
    embedding=hf_em_model

)

<ipython-input-17-717068d55aa3>:5: LangChainDeprecationWarning: The class `HuggingFaceBgeEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  hf_em_model=HuggingFaceBgeEmbeddings(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
import os
os.environ["GROQ_API_KEY"]=""

In [ ]:
## multi query
from langchain_groq import ChatGroq
from langchain.prompts import ChatPromptTemplate
template="""
You are an AI assistant that generate multiple question based on the user's input
query or question. Generate multiple queries such that the questions are caputured the
what the question is asking about in a different perspective.
generate 5 queries in new line each based on user input query:
{question}
output 5 questions

"""
prompt=ChatPromptTemplate.from_template(template)

llm=ChatGroq(
    model="llama3-8b-8192",
    temperature=0
)

generate_queries_chain=(
    prompt
    |llm
    |(lambda x: x.content.split("\n"))
)






In [ ]:
question="what is llm?"
questions=generate_queries_chain.invoke(question)
questions

['Here are 5 questions that capture the essence of the user\'s query "what is LLM?" from different perspectives:',
 '',
 'What is the primary function of a Large Language Model (LLM) in natural language processing?',
 '',
 'How does a Large Language Model (LLM) differ from other types of language models in terms of its architecture and capabilities?',
 '',
 'What are some of the key applications and use cases of Large Language Models (LLMs) in industries such as customer service, content creation, and language translation?',
 '',
 'Can a Large Language Model (LLM) be trained to perform specific tasks such as text classification, sentiment analysis, and question answering, and if so, how?',
 '',
 'What are the potential limitations and challenges of using Large Language Models (LLMs) in real-world applications, and how can these be addressed through further research and development?']